# Rag Engine

In [2]:
# !pip install -U --user --quiet google-cloud-aiplatform google-cloud-discoveryengine
# !pip install -U google-genai

In [4]:
import os
from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine
from vertexai.preview import rag
import vertexai

PROJECT_ID = "hello-world-418507"
LOCATION = "us-central1"
VERTEX_AI_SEARCH_LOCATION = "global"
vertexai.init(project=PROJECT_ID, location=LOCATION)

## Configuration

In [5]:
# The datastore name can only contain lowercase letters, numbers, and hyphens
DATASTORE_NAME = "2025-test-datastore-name"
DATASTORE_ID = f"{DATASTORE_NAME}-id"

GCS_BUCKET = "gs://2025-traveloka-gds"


# Part 1 - Datastore (Run once only)

### Create and Populate a Datastore

RAG is not supported for the Vertex AI Search DataStore with 'chunking config'.

In [4]:
def create_data_store(
    project_id: str, location: str, data_store_name: str, data_store_id: str
):
    # Create a client
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = discoveryengine.DataStoreServiceClient(client_options=client_options)

    # Initialize request argument(s)
    data_store = discoveryengine.DataStore(
        display_name=data_store_name,
        industry_vertical=discoveryengine.IndustryVertical.GENERIC,
        content_config=discoveryengine.DataStore.ContentConfig.CONTENT_REQUIRED,
    )

    operation = client.create_data_store(
        request=discoveryengine.CreateDataStoreRequest(
            parent=client.collection_path(project_id, location, "default_collection"),
            data_store=data_store,
            data_store_id=data_store_id,
        )
    )

    # Make the request
    response = operation.result(timeout=90)
    return response.name

In [7]:
def import_documents(
    project_id: str,
    location: str,
    data_store_id: str,
    gcs_uri: str, # Bucket path
):
    # Create a client
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = discoveryengine.DocumentServiceClient(client_options=client_options)

    # The full resource name of the search engine branch.
    # e.g. projects/{project}/locations/{location}/dataStores/{data_store_id}/branches/{branch}
    parent = client.branch_path(
        project=project_id,
        location=location,
        data_store=data_store_id,
        branch="default_branch",
    )

    source_documents = [f"{gcs_uri}/*"]

    request = discoveryengine.ImportDocumentsRequest(
        parent=parent,
        gcs_source=discoveryengine.GcsSource(
            input_uris=source_documents, data_schema="content"
        ),
        # Options: `FULL`, `INCREMENTAL`
        reconciliation_mode=discoveryengine.ImportDocumentsRequest.ReconciliationMode.INCREMENTAL,
    )

    # Make the request
    operation = client.import_documents(request=request)

    response = operation.result()

    # Once the operation is complete,
    # get information from operation metadata
    metadata = discoveryengine.ImportDocumentsMetadata(operation.metadata)

    # Handle the response
    return operation.operation.name

In [5]:
create_data_store(PROJECT_ID, VERTEX_AI_SEARCH_LOCATION, DATASTORE_NAME, DATASTORE_ID)

'projects/671247654914/locations/global/collections/default_collection/dataStores/2025-test-datastore-name-id'

In [ ]:
import_documents(PROJECT_ID, VERTEX_AI_SEARCH_LOCATION, DATASTORE_ID, GCS_BUCKET)

## Part 2 - Create a Search Engine (Run once only)

In [11]:
def create_engine(
    project_id: str, location: str, engine_name: str, engine_id: str, data_store_id: str
):
    # Create a client
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = discoveryengine.EngineServiceClient(client_options=client_options)

    # Initialize request argument(s)
    engine = discoveryengine.Engine(
        display_name=engine_name,
        solution_type=discoveryengine.SolutionType.SOLUTION_TYPE_SEARCH,
        industry_vertical=discoveryengine.IndustryVertical.GENERIC,
        data_store_ids=[data_store_id],
        search_engine_config=discoveryengine.Engine.SearchEngineConfig(
            search_tier=discoveryengine.SearchTier.SEARCH_TIER_ENTERPRISE,
        ),
    )
    print(engine.display_name)

    request = discoveryengine.CreateEngineRequest(
        parent=client.collection_path(project_id, location, "default_collection"),
        engine=engine,
        engine_id=engine_id,
    )

    # Make the request
    operation = client.create_engine(request=request)
    response = operation.result(timeout=90)
    return response.name

In [12]:
DATASTORE_NAME, DATASTORE_ID

('2025-test-datastore-name', '2025-test-datastore-name-id')

In [13]:
ENGINE_NAME = DATASTORE_NAME + "-engine"
ENGINE_ID = DATASTORE_ID + "-engine"

response = create_engine(
    project_id=PROJECT_ID,
    location=VERTEX_AI_SEARCH_LOCATION,
    engine_name=ENGINE_NAME,
    engine_id=ENGINE_ID,
    data_store_id=DATASTORE_ID,
)

response

2025-test-datastore-name-engine


'projects/671247654914/locations/global/collections/default_collection/engines/2025-test-datastore-name-id-engine'

## Part 3 - Create corpus

In [6]:
from vertexai.preview import rag

In [ ]:
VERTEX_AI_SEARCH_LOCATION, ENGINE_NAME, ENGINE_ID

In [33]:
"""
Format:
    ``projects/{project}/locations/{location}/collections/{collection}/engines/{engine_id}/servingConfigs/{serving_config}``
or
    ``projects/{project}/locations/{location}/collections/{collection}/dataStores/{data_store_id}/servingConfigs/{serving_config}``
"""

config = f"projects/{PROJECT_ID}/locations/{VERTEX_AI_SEARCH_LOCATION}/collections/default_collection/engines/{ENGINE_ID}/servingConfigs/default_search"
print(config)

vertex_ai_search_config = rag.VertexAiSearchConfig(
    serving_config=config,
)


projects/hello-world-418507/locations/global/collections/default_collection/engines/2025-test-datastore-name-id-engine/servingConfigs/default_search


In [34]:
rag_corpus = rag.create_corpus(
    display_name=ENGINE_NAME,
    vertex_ai_search_config=vertex_ai_search_config,
)


In [35]:
print(rag_corpus.name)

projects/671247654914/locations/us-central1/ragCorpora/1113515007867355136


In [36]:
# Check the corpus just created
new_corpus = rag.get_corpus(name=rag_corpus.name)
new_corpus

RagCorpus(name='projects/671247654914/locations/us-central1/ragCorpora/1113515007867355136', display_name='2025-test-datastore-name-engine', description='', embedding_model_config=EmbeddingModelConfig(publisher_model=None, endpoint=None, model=None, model_version_id=None), vector_db=None, vertex_ai_search_config=VertexAiSearchConfig(serving_config='projects/hello-world-418507/locations/global/collections/default_collection/engines/2025-test-datastore-name-id-engine/servingConfigs/default_search'), backend_config=RagVectorDbConfig(vector_db=None, rag_embedding_model_config=None))

## List corpora

In [7]:
rag.list_corpora()

ListRagCorporaPager<rag_corpora {
  name: "projects/hello-world-418507/locations/us-central1/ragCorpora/1113515007867355136"
  display_name: "2025-test-datastore-name-engine"
  create_time {
    seconds: 1742560106
    nanos: 86801000
  }
  update_time {
    seconds: 1742560106
    nanos: 86801000
  }
  corpus_status {
    state: ACTIVE
  }
  vertex_ai_search_config {
    serving_config: "projects/hello-world-418507/locations/global/collections/default_collection/engines/2025-test-datastore-name-id-engine/servingConfigs/default_search"
  }
}
rag_corpora {
  name: "projects/hello-world-418507/locations/us-central1/ragCorpora/4611686018427387904"
  display_name: "Test_corpus"
  description: "Corpus_for_simple_test."
  create_time {
    seconds: 1743424713
    nanos: 154943000
  }
  update_time {
    seconds: 1743424713
    nanos: 154943000
  }
  rag_embedding_model_config {
    vertex_prediction_endpoint {
      endpoint: "projects/hello-world-418507/locations/us-central1/publishers/goog

# Part 4 - Retrieval

### Query with Rag Engine

In [8]:
RETRIEVAL_QUERY = "How to do full refund for unused ticket?"

rag_resource = rag.RagResource(rag_corpus="projects/671247654914/locations/us-central1/ragCorpora/1113515007867355136")
print(rag_resource)

rag_retrieval_config = rag.RagRetrievalConfig(
    top_k=5,  # Optional
    filter=rag.Filter(
        vector_distance_threshold=0.2,  # Optional
    ),
)

response = rag.retrieval_query(
    rag_resources=[rag_resource],  # Currently only 1 corpus is allowed.
    text=RETRIEVAL_QUERY,
    similarity_top_k=10,
    # rag_retrieval_config=rag_retrieval_config
)
print(response)


RagResource(rag_corpus='projects/671247654914/locations/us-central1/ragCorpora/1113515007867355136', rag_file_ids=None)
contexts {
  contexts {
    source_uri: "gs://2025-traveloka-gds/FOD-[INTERNAL] Sabre - Manual Refund Fully Unused Ticket-040325-042757.pdf"
    text: "Once you do, the following displays: ETR REFUND MESSAGE PROCESSED REA ISSUE N/A Check refunded amount There are several ways to check the refund amount once the ticket is refunded in GDS, ensure you are in the correct PCC to be able to pull out the correct information."
    source_display_name: "[INTERNAL] Sabre - Manual Refund Fully Unused Ticket - Flight Ops Knowledge Base - Confluence"
  }
  contexts {
    source_uri: "gs://2025-traveloka-gds/Electronic Ticket - Sabre Fares & Ticketing Guide.pdf"
    text: "ETR REFUND Refund untuk Electronic Ticket dapat dilakukan melalui system Abacus, tidak perlu melakukan refund secara manual. Refund dapat dilakukan dengan 2 cara : a. Current PNR b. Past Date PNR Step – step untu

## Query with Gemini


In [9]:
from rich import print as rich_print

from google.genai.types import (
    GenerateContentConfig,
    VertexRagStore,
    Retrieval,
    Tool,
    VertexAISearch,
)
from google import genai


In [10]:

PROJECT_ID = "hello-world-418507"
LOCATION = "us-central1"
MODEL_ID = "gemini-2.0-flash-001"  # @param {type: "string"}
RETRIEVAL_QUERY = "How to do full refund for unused ticket?"


client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

rag_resource = rag.RagResource(
    rag_corpus="projects/hello-world-418507/locations/us-central1/ragCorpora/1113515007867355136",
)

vertex_ai_search_tool = Tool(
    retrieval=Retrieval(vertex_rag_store=VertexRagStore(rag_resources=[rag_resource]))
)

response = client.models.generate_content(
    model=MODEL_ID,
    contents=RETRIEVAL_QUERY,
    config=GenerateContentConfig(tools=[vertex_ai_search_tool]),
)

rich_print(response)


GenerateContentResponse(
    candidates=[
        Candidate(
            content=Content(
                parts=[
                    Part(
                        video_metadata=None,
                        thought=None,
                        code_execution_result=None,
                        executable_code=None,
                        file_data=None,
                        function_call=None,
                        function_response=None,
                        inline_data=None,
                        text='In Sabre, you can refund an electronic ticket (ETR) through the system without manual
processing. This can be done for both current and past date Passenger Name Records (PNR).\n\nTo refund a current 
PNR:\n\n1.  Retrieve the PNR.\n2.  Enter \\*T.\n3.  Enter WETR\\*(LINE NUMBER) (e.g., WETR\\*2).\n\nAfter refunding
a ticket in GDS, you can check the refunded amount. Once processed, the system will display: ETR REFUND MESSAGE 
PROCESSED REA ISSUE N/A. Ensure you are in the correct PCC to pull the correct information.\n'
                    )
                ],
                role='model'
            ),
            citation_metadata=None,
            finish_message=None,
            token_count=None,
            finish_reason=<FinishReason.STOP: 'STOP'>,
            avg_logprobs=-0.13706814801251446,
            grounding_metadata=GroundingMetadata(
                grounding_chunks=[
                    GroundingChunk(
                        retrieved_context=GroundingChunkRetrievedContext(
                            text='Once you do, the following displays: ETR REFUND MESSAGE PROCESSED REA ISSUE N/A 
Check refunded amount There are several ways to check the refund amount once the ticket is refunded in GDS, ensure 
you are in the correct PCC to be able to pull out the correct information.',
                            title='[INTERNAL] Sabre - Manual Refund Fully Unused Ticket - Flight Ops Knowledge Base
- Confluence',
                            uri='gs://2025-traveloka-gds/FOD-[INTERNAL] Sabre - Manual Refund Fully Unused 
Ticket-040325-042757.pdf'
                        ),
                        web=None
                    ),
                    GroundingChunk(
                        retrieved_context=GroundingChunkRetrievedContext(
                            text='ETR REFUND Refund untuk Electronic Ticket dapat dilakukan melalui system Abacus, 
tidak perlu melakukan refund secara manual. Refund dapat dilakukan dengan 2 cara : a. Current PNR b. Past Date PNR 
Step – step untuk Current PNR 1. Retrieve PNR 2. *T 3. WETR*(LINE NUMBER) ex. WETR*2 4.',
                            title='Microsoft Word - Electronic Ticket - Sabre Fares & Ticketing Guide',
                            uri='gs://2025-traveloka-gds/Electronic Ticket - Sabre Fares & Ticketing Guide.pdf'
                        ),
                        web=None
                    ),
                    GroundingChunk(
                        retrieved_context=GroundingChunkRetrievedContext(
                            text='Some alpha codes represent individual categories, while others group rules 
together by topic. You can use any of the alpha codes together, up to a maximum of five alpha codes (not counting 
the IC code), in a single entry.',
                            title='How to Understand Fare Rule Categories',
                            uri='gs://2025-traveloka-gds/How to Understand Fare Rule Categories.pdf'
                        ),
                        web=None
                    ),
                    GroundingChunk(
                        retrieved_context=GroundingChunkRetrievedContext(
                            text='How long does Sabre hold the ticket image in the ticket image database?',
                            title='Display the Database Ticket Image (WTDB*)',
                            uri='gs://2025-traveloka-gds/Display the Database Ticket Image (WTDB_).pdf'
                        ),
                        w

## Cleaning up

Clean up RAG resources created in this notebook.

In [45]:

rag.delete_corpus(name="projects/hello-world-418507/locations/us-central1/ragCorpora/6838716034162098176")  # pragma: allowlist secret

Successfully deleted the RagCorpus.
